In [1]:
import numpy as np

E = 210e9
Largo = 6
I = 2e-4
MC = [(0,1),(1,2)]
Elementos = len(MC)
k_resorte = 200000
    

#1 es condicion de fuerza, 0 es condicion de angulo o desplazamiento
Vinculos = [(0,0), # x nodo 0
            (0,0), # y nodo 0 
            (0,0), # x nodo 1
            (1,0), # y nodo 1 
            (1,-50000), # x nodo 2
            (1,0), # y nodo 2            
            (0,0), # x nodo 3
            (0,0)] # y nodo 3

In [2]:

L = Largo/Elementos
K = (E*I)/(L**3)
n = (Elementos+1)*2 + 2 
Matriz_global = np.zeros([n,n])


Desplazamientos_f = np.zeros(n)
Fuerza = []
Nodos = []

In [3]:
# Resolver primero lo demas menos el resorte:

for i in range(Elementos):
    
     p,j = MC[i]
    
     Matriz = np.array([[12, 6*L, -12, 6*L],
                       [6*L, 4*L**2, -6*L, 2*L**2],
                       [-12 , -6*L, 12, -6*L],
                       [6*L,2*L**2,-6*L, 4* L**2]])
    
     componente = K*Matriz

     A_G = np.array([p*2,p*2+1],dtype = int)
     B_G = np.array([j*2,j*2+1],dtype = int) 
    
     A_L = np.array([0,1],dtype = int)
     B_L = np.array([2,3],dtype = int)

    
     Matriz_global[np.ix_(A_G,A_G)] +=  componente [np.ix_(A_L,A_L)]
     Matriz_global[np.ix_(B_G,B_G)] +=  componente [np.ix_(B_L,B_L)]
     Matriz_global[np.ix_(A_G,B_G)] +=  componente [np.ix_(A_L,B_L)]
     Matriz_global[np.ix_(B_G,A_G)] +=  componente [np.ix_(B_L,A_L)]



[[ 0.33333333  0.5        -0.33333333  0.5       ]
 [ 0.5         1.         -0.5         0.5       ]
 [-0.33333333 -0.5         0.33333333 -0.5       ]
 [ 0.5         0.5        -0.5         1.        ]]
56000000.0
[[ 0.33333333  0.5        -0.33333333  0.5       ]
 [ 0.5         1.         -0.5         0.5       ]
 [-0.33333333 -0.5         0.33333333 -0.5       ]
 [ 0.5         0.5        -0.5         1.        ]]
56000000.0


In [4]:

Matriz_resorte = np.array([[1,0,-1,0],
                           [0,0,0,0],
                           [-1,0,1,0],
                           [0,0,0,0]])

componente_resorte = k_resorte * Matriz_resorte
resorte_1 = np.array([n-4,n-3],dtype = int)
resorte_2 = np.array([n-2,n-1],dtype = int) 
    
A_L = np.array([0,1],dtype = int)
B_L = np.array([2,3],dtype = int)

    
Matriz_global[np.ix_(resorte_1,resorte_1)] +=  componente_resorte [np.ix_(A_L,A_L)]
Matriz_global[np.ix_(resorte_2,resorte_2)] +=  componente_resorte [np.ix_(B_L,B_L)]
Matriz_global[np.ix_(resorte_1,resorte_2)] +=  componente_resorte [np.ix_(A_L,B_L)]
Matriz_global[np.ix_(resorte_2,resorte_1)] +=  componente_resorte [np.ix_(B_L,A_L)]
   

In [5]:

for i in range(len(Vinculos)):
    tipo,valor = Vinculos[i]  
    if tipo == 1:
        Fuerza.append(valor)
        Nodos.append(i)
    if tipo == 0:
        Desplazamientos_f[i] = valor

Fuerza = np.array(Fuerza).reshape(-1,1)

Matriz_res = Matriz_global[np.ix_(Nodos,Nodos)]

Desplazamientos_parcial = np.linalg.solve(Matriz_res,Fuerza)


o = 0
for i in Nodos:
    Desplazamientos_f[i] = Desplazamientos_parcial [o]
    o += 1
  

Fuerzas_ = np.dot(Matriz_global,Desplazamientos_f)
print(Fuerzas_.round(4))
print()
print(Desplazamientos_f.round(4))

[-69767.4419 -69767.4419 116279.0698      0.     -50000.         -0.
   3488.3721      0.    ]

[ 0.      0.      0.     -0.0025 -0.0174 -0.0075  0.      0.    ]


In [7]:

np.set_printoptions(linewidth = 700, precision = 4 )

print(Matriz_global/Matriz_global.max())
print(Matriz_global.max())

[[ 0.1667  0.25   -0.1667  0.25    0.      0.      0.      0.    ]
 [ 0.25    0.5    -0.25    0.25    0.      0.      0.      0.    ]
 [-0.1667 -0.25    0.3333  0.     -0.1667  0.25    0.      0.    ]
 [ 0.25    0.25    0.      1.     -0.25    0.25    0.      0.    ]
 [ 0.      0.     -0.1667 -0.25    0.1685 -0.25   -0.0018  0.    ]
 [ 0.      0.      0.25    0.25   -0.25    0.5     0.      0.    ]
 [ 0.      0.      0.      0.     -0.0018  0.      0.0018  0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.    ]]
112000000.0
